# CBOW와 단어 임베딩 구현하기

이 노트북에서는 CBOW(Continuous Bag of Words) 모델을 구현하여 단어 임베딩을 학습하는 방법을 배워보겠습니다.

CBOW 모델은 Word2Vec의 한 종류로, 주변 단어들(컨텍스트)을 사용하여 중심 단어를 예측하는 방식으로 단어의 밀집 표현(dense representation)을 학습합니다.

## 주요 학습 내용:
- 텍스트 전처리와 어휘집 구성
- 신경망 기반 언어 모델 구현
- 순전파와 역전파 알고리즘
- 경사 하강법을 통한 최적화
- 학습된 임베딩의 시각화


## 1. 필요한 라이브러리 import


In [ ]:
import nltk
import numpy as np
from utils import get_batches
from utils import compute_pca
from utils import get_dict
import re
from matplotlib import pyplot as plt

# NLTK 다운로드 (필요한 경우)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("NLTK punkt 토크나이저를 다운로드합니다...")
    nltk.download('punkt')

print("필요한 라이브러리를 성공적으로 import했습니다.")


## 2. 데이터 로드 및 전처리

햄릿 텍스트를 로드하고 CBOW 모델 훈련에 적합하도록 전처리합니다.


In [ ]:
# 햄릿 텍스트 로드
try:
    with open("../../data/hamlet.txt", "r", encoding="utf-8") as f:
        data = f.read()
    print("햄릿 텍스트를 성공적으로 로드했습니다.")
    print(f"원본 텍스트 길이: {len(data):,} 문자")
    print(f"처음 200자: {data[:200]}...")
except FileNotFoundError:
    print("햄릿 파일을 찾을 수 없습니다. 샘플 데이터로 대체합니다.")
    data = """
    To be or not to be, that is the question. Whether 'tis nobler in the mind to suffer 
    the slings and arrows of outrageous fortune, or to take arms against a sea of troubles 
    and by opposing end them. To die, to sleep, no more, and by a sleep to say we end 
    the heartache and the thousand natural shocks that flesh is heir to.
    """ * 100  # 더 많은 데이터를 위해 반복

print(f"로드된 데이터 길이: {len(data):,} 문자")


In [ ]:
# 텍스트 전처리
print("텍스트 전처리를 시작합니다...")

# 1단계: 구두점을 마침표로 통일
print("1단계: 구두점 정리")
print(f"전처리 전: {data[500:600]}")
data = re.sub(r"[,!?;-]", ".", data)
print(f"전처리 후: {data[500:600]}")

# 2단계: 단어 단위로 토큰화
print("\\n2단계: 토큰화")
data = nltk.word_tokenize(data)
print(f"토큰화 후 처음 20개 토큰: {data[:20]}")

# 3단계: 알파벳 단어만 유지하고 소문자로 변환
print("\\n3단계: 알파벳 필터링 및 소문자 변환")
original_length = len(data)
data = [ch.lower() for ch in data if ch.isalpha() or ch == "."]
print(f"필터링 전 토큰 수: {original_length:,}")
print(f"필터링 후 토큰 수: {len(data):,}")
print(f"샘플 토큰들 (500-515): {data[500:515]}")

print(f"\\n✅ 전처리 완료! 총 {len(data):,}개의 토큰")


## 3. 어휘집 구성 및 빈도 분석

텍스트에서 어휘집을 추출하고 단어 빈도를 분석합니다.


In [ ]:
# 단어 빈도 분석
print("단어 빈도 분석을 수행합니다...")
fdist = nltk.FreqDist(word for word in data)

print(f"어휘집 크기: {len(fdist):,}개 고유 단어")
print(f"전체 토큰 수: {sum(fdist.values()):,}개")

# 가장 빈번한 단어들 확인
print("\\n📊 가장 빈번한 20개 단어:")
most_common = fdist.most_common(20)
for i, (word, freq) in enumerate(most_common, 1):
    print(f"  {i:2d}. '{word}': {freq:,}회 ({freq/len(data)*100:.2f}%)")

# 빈도 분포 시각화 준비
words, frequencies = zip(*most_common)
print(f"\\n상위 10개 단어가 전체의 {sum(frequencies[:10])/len(data)*100:.1f}%를 차지")


## 4. 단어-인덱스 매핑 딕셔너리 생성

효율적인 처리를 위해 단어와 인덱스 간의 양방향 매핑을 생성합니다.


In [ ]:
# 단어-인덱스 매핑 딕셔너리 생성
print("단어-인덱스 매핑 딕셔너리를 생성합니다...")
word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)

print(f"어휘집 크기 (V): {V:,}")
print(f"word2Ind 딕셔너리 타입: {type(word2Ind)}")
print(f"Ind2word 딕셔너리 타입: {type(Ind2word)}")

# 매핑 예시 확인
test_words = ['king', 'queen', 'the', 'to', 'be']
print("\\n단어 → 인덱스 매핑 예시:")
for word in test_words:
    if word in word2Ind:
        idx = word2Ind[word]
        print(f"  '{word}' → {idx}")
        # 역매핑도 확인
        reverse_word = Ind2word[idx]
        print(f"  {idx} → '{reverse_word}' ✓")
    else:
        print(f"  '{word}' → 어휘집에 없음")

# 특정 인덱스들의 단어 확인
print("\\n인덱스 → 단어 매핑 예시:")
sample_indices = [0, 1, 100, 500, 1000]
for idx in sample_indices:
    if idx < V:
        word = Ind2word[idx]
        print(f"  인덱스 {idx} → '{word}'")


## 5. 신경망 모델 초기화

CBOW 모델을 위한 신경망 가중치와 편향을 초기화합니다.


In [ ]:
def initialize_model(N, V, random_seed=1):
    """
    CBOW 모델의 가중치와 편향을 초기화합니다.
    
    Args:
        N: 은닉층(임베딩) 벡터의 차원
        V: 어휘집 크기
        random_seed: 재현 가능한 결과를 위한 시드
    
    Returns:
        W1: 입력층 → 은닉층 가중치 (N × V)
        W2: 은닉층 → 출력층 가중치 (V × N)  
        b1: 은닉층 편향 (N × 1)
        b2: 출력층 편향 (V × 1)
    """
    np.random.seed(random_seed)
    
    # 가중치를 0과 1 사이의 난수로 초기화
    W1 = np.random.rand(N, V)
    W2 = np.random.rand(V, N)
    b1 = np.random.rand(N, 1)
    b2 = np.random.rand(V, 1)
    
    return W1, W2, b1, b2

# 모델 파라미터 설정
N = 50  # 임베딩 차원
print(f"모델 설정:")
print(f"  임베딩 차원 (N): {N}")
print(f"  어휘집 크기 (V): {V:,}")

# 모델 초기화 테스트
W1, W2, b1, b2 = initialize_model(N, V, random_seed=42)

print(f"\\n초기화된 파라미터 크기:")
print(f"  W1 (입력→은닉): {W1.shape}")
print(f"  W2 (은닉→출력): {W2.shape}")
print(f"  b1 (은닉층 편향): {b1.shape}")
print(f"  b2 (출력층 편향): {b2.shape}")

# 전체 파라미터 수 계산
total_params = W1.size + W2.size + b1.size + b2.size
print(f"\\n전체 파라미터 수: {total_params:,}개")

# 가중치 값 분포 확인
print(f"\\nW1 통계: min={W1.min():.4f}, max={W1.max():.4f}, mean={W1.mean():.4f}")
print(f"W2 통계: min={W2.min():.4f}, max={W2.max():.4f}, mean={W2.mean():.4f}")


## 6. 활성화 함수: Softmax 구현

출력층에서 확률 분포를 생성하기 위한 Softmax 함수를 구현합니다.


In [ ]:
def softmax(z):
    """
    Softmax 활성화 함수
    모든 출력의 합이 1이 되도록 정규화하여 확률 분포를 생성
    
    Args:
        z: 은닉층의 출력 점수 (V × batch_size)
    
    Returns:
        yhat: 확률 분포 (각 단어에 대한 예측 확률)
    """
    # 수치적 안정성을 위해 최대값을 빼줌 (exp 오버플로우 방지)
    z_shifted = z - np.max(z, axis=0, keepdims=True)
    exp_z = np.exp(z_shifted)
    yhat = exp_z / np.sum(exp_z, axis=0, keepdims=True)
    return yhat

# Softmax 함수 테스트
print("Softmax 함수 테스트:")

# 간단한 테스트 벡터
test_z = np.array([[2.0], [1.0], [0.1]])
test_probs = softmax(test_z)

print(f"입력 z: {test_z.flatten()}")
print(f"출력 확률: {test_probs.flatten()}")
print(f"확률 합: {np.sum(test_probs):.6f} (1에 가까워야 함)")

# 더 복잡한 테스트 (배치 처리)
print("\\n배치 처리 테스트:")
batch_z = np.random.randn(5, 3)  # 5개 단어, 3개 샘플
batch_probs = softmax(batch_z)

print(f"배치 입력 크기: {batch_z.shape}")
print(f"배치 출력 크기: {batch_probs.shape}")
print(f"각 샘플의 확률 합: {np.sum(batch_probs, axis=0)}")

# 극단적인 값에서의 안정성 테스트
print("\\n수치적 안정성 테스트:")
extreme_z = np.array([[100.0], [200.0], [300.0]])
extreme_probs = softmax(extreme_z)
print(f"극단적 입력: {extreme_z.flatten()}")
print(f"안정적 출력: {extreme_probs.flatten()}")
print(f"무한대나 NaN 없음: {np.all(np.isfinite(extreme_probs))}")


## 7. 순전파(Forward Propagation) 구현

입력에서 출력까지 신경망을 통과하는 순전파 과정을 구현합니다.


In [ ]:
def forward_prop(x, W1, W2, b1, b2):
    """
    CBOW 모델의 순전파
    
    Args:
        x: 컨텍스트 단어들의 평균 원핫 벡터 (V × batch_size)
        W1, W2, b1, b2: 모델 파라미터들
    
    Returns:
        z: 출력층의 점수 벡터 (V × batch_size)
        h: 은닉층 벡터 (활성화 후, N × batch_size)
    """
    # 1단계: 입력층 → 은닉층
    h_raw = W1 @ x + b1  # 선형 변환
    h = np.maximum(0, h_raw)  # ReLU 활성화 함수
    
    # 2단계: 은닉층 → 출력층
    z = W2 @ h + b2  # 선형 변환 (Softmax는 나중에 적용)
    
    return z, h

# 순전파 테스트
print("순전파 함수 테스트:")

# 가짜 입력 데이터 생성 (원핫 벡터)
batch_size = 3
x_test = np.zeros((V, batch_size))

# 몇 개 단어에 대해 원핫 인코딩 (예시)
test_word_indices = [10, 25, 50]
for i, word_idx in enumerate(test_word_indices):
    if word_idx < V:
        x_test[word_idx, i] = 1.0

print(f"테스트 입력 크기: {x_test.shape}")
print(f"입력의 원핫 검증: {np.sum(x_test, axis=0)} (각각 1이어야 함)")

# 순전파 실행
z_test, h_test = forward_prop(x_test, W1, W2, b1, b2)

print(f"\\n순전파 결과:")
print(f"은닉층 출력 (h) 크기: {h_test.shape}")
print(f"출력층 점수 (z) 크기: {z_test.shape}")

# 은닉층 통계
print(f"\\n은닉층 (h) 통계:")
print(f"  최소값: {h_test.min():.4f}")
print(f"  최대값: {h_test.max():.4f}")
print(f"  평균값: {h_test.mean():.4f}")
print(f"  ReLU로 인한 0 개수: {np.sum(h_test == 0)}")

# 출력층 통계
print(f"\\n출력층 (z) 통계:")
print(f"  최소값: {z_test.min():.4f}")
print(f"  최대값: {z_test.max():.4f}")
print(f"  평균값: {z_test.mean():.4f}")

# 확률로 변환해보기
probs_test = softmax(z_test)
print(f"\\nSoftmax 후 확률:")
print(f"  확률 합: {np.sum(probs_test, axis=0)}")
print(f"  최고 확률들: {np.max(probs_test, axis=0)}")


## 8. 비용 함수(Cost Function) 구현

모델의 예측과 실제 값 사이의 차이를 측정하는 비용 함수를 구현합니다.


In [ ]:
def compute_cost(y, yhat, batch_size):
    """
    Cross-entropy 비용 함수
    
    Args:
        y: 실제 레이블 (원핫 벡터, V × batch_size)
        yhat: 예측 확률 (V × batch_size)
        batch_size: 배치 크기
    
    Returns:
        cost: 평균 cross-entropy 손실
    """
    # 수치적 안정성을 위해 log(0) 방지
    epsilon = 1e-15
    yhat_clipped = np.clip(yhat, epsilon, 1 - epsilon)
    
    # Cross-entropy 계산
    logprobs = np.multiply(np.log(yhat_clipped), y) + np.multiply(
        np.log(1 - yhat_clipped), 1 - y
    )
    
    # 평균 손실 계산
    cost = -1 / batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)  # 스칼라로 변환
    
    return cost

# 비용 함수 테스트
print("비용 함수 테스트:")

# 완벽한 예측 케이스 (cost ≈ 0)
print("\\n1. 완벽한 예측 케이스:")
y_perfect = np.array([[1.0, 0.0], [0.0, 1.0], [0.0, 0.0]])  # 3×2
yhat_perfect = np.array([[0.99, 0.01], [0.01, 0.99], [0.0, 0.0]])  # 3×2
cost_perfect = compute_cost(y_perfect, yhat_perfect, 2)
print(f"실제: {y_perfect.T}")
print(f"예측: {yhat_perfect.T}")
print(f"비용: {cost_perfect:.6f} (0에 가까워야 함)")

# 최악의 예측 케이스 (cost 높음)
print("\\n2. 최악의 예측 케이스:")
y_worst = np.array([[1.0, 0.0], [0.0, 1.0], [0.0, 0.0]])  # 3×2
yhat_worst = np.array([[0.01, 0.99], [0.99, 0.01], [0.0, 0.0]])  # 3×2
cost_worst = compute_cost(y_worst, yhat_worst, 2)
print(f"실제: {y_worst.T}")
print(f"예측: {yhat_worst.T}")
print(f"비용: {cost_worst:.6f} (높아야 함)")

# 랜덤 예측 케이스
print("\\n3. 랜덤 예측 케이스:")
np.random.seed(42)
y_random = np.array([[1.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 0.0, 0.0]])  # 3×3
yhat_random = np.random.rand(3, 3)
yhat_random = yhat_random / np.sum(yhat_random, axis=0)  # 확률로 정규화
cost_random = compute_cost(y_random, yhat_random, 3)
print(f"실제 첫 번째 샘플: {y_random[:, 0]}")
print(f"예측 첫 번째 샘플: {yhat_random[:, 0]}")
print(f"비용: {cost_random:.6f}")

# 실제 모델 출력으로 테스트
print("\\n4. 실제 모델 출력으로 테스트:")
# 앞서 생성한 순전파 결과 사용
y_actual = np.zeros((V, batch_size))
target_indices = [100, 200, 300]  # 임의의 타겟 단어들
for i, target_idx in enumerate(target_indices):
    if target_idx < V:
        y_actual[target_idx, i] = 1.0

yhat_actual = softmax(z_test)
cost_actual = compute_cost(y_actual, yhat_actual, batch_size)
print(f"실제 모델 비용: {cost_actual:.6f}")


## 9. 역전파(Backpropagation) 구현

비용을 최소화하기 위해 그래디언트를 계산하는 역전파를 구현합니다.


In [ ]:
def back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size):
    """
    역전파를 통한 그래디언트 계산
    
    Args:
        x: 입력 (평균 원핫 벡터, V × batch_size)
        yhat: 예측 확률 (V × batch_size)
        y: 실제 레이블 (V × batch_size)
        h: 은닉층 출력 (N × batch_size)
        W1, W2, b1, b2: 모델 파라미터들
        batch_size: 배치 크기
    
    Returns:
        grad_W1, grad_W2, grad_b1, grad_b2: 각 파라미터의 그래디언트
    """
    # 출력층 오차 (V × batch_size)
    output_error = yhat - y
    
    # 은닉층으로의 역전파 (ReLU 활성화 고려)
    l1 = np.dot(W2.T, output_error)  # (N × batch_size)
    l1 = np.maximum(0, l1)  # ReLU 미분 (h > 0인 곳만 통과)
    
    # 그래디언트 계산
    grad_W1 = np.dot(l1, x.T) / batch_size  # (N × V)
    grad_W2 = np.dot(output_error, h.T) / batch_size  # (V × N)
    grad_b1 = np.sum(l1, axis=1, keepdims=True) / batch_size  # (N × 1)
    grad_b2 = np.sum(output_error, axis=1, keepdims=True) / batch_size  # (V × 1)
    
    return grad_W1, grad_W2, grad_b1, grad_b2

# 역전파 테스트
print("역전파 함수 테스트:")

# 앞서 생성한 순전파 결과 활용
yhat_test = softmax(z_test)

# 가짜 타겟 생성
y_test = np.zeros((V, batch_size))
target_words = [word2Ind.get('king', 0), word2Ind.get('queen', 1), word2Ind.get('the', 2)]
for i, target_idx in enumerate(target_words):
    y_test[target_idx, i] = 1.0

print(f"테스트 설정:")
print(f"  입력 크기: {x_test.shape}")
print(f"  예측 크기: {yhat_test.shape}")
print(f"  타겟 크기: {y_test.shape}")
print(f"  은닉층 크기: {h_test.shape}")

# 역전파 실행
grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(
    x_test, yhat_test, y_test, h_test, W1, W2, b1, b2, batch_size
)

print(f"\\n계산된 그래디언트 크기:")
print(f"  grad_W1: {grad_W1.shape}")
print(f"  grad_W2: {grad_W2.shape}")
print(f"  grad_b1: {grad_b1.shape}")
print(f"  grad_b2: {grad_b2.shape}")

print(f"\\n그래디언트 통계:")
print(f"  grad_W1: min={grad_W1.min():.6f}, max={grad_W1.max():.6f}, mean={grad_W1.mean():.6f}")
print(f"  grad_W2: min={grad_W2.min():.6f}, max={grad_W2.max():.6f}, mean={grad_W2.mean():.6f}")
print(f"  grad_b1: min={grad_b1.min():.6f}, max={grad_b1.max():.6f}, mean={grad_b1.mean():.6f}")
print(f"  grad_b2: min={grad_b2.min():.6f}, max={grad_b2.max():.6f}, mean={grad_b2.mean():.6f}")

# 그래디언트가 0이 아닌지 확인 (학습이 일어나는지 확인)
print(f"\\n그래디언트 0이 아닌 원소 비율:")
print(f"  grad_W1: {np.mean(grad_W1 != 0)*100:.1f}%")
print(f"  grad_W2: {np.mean(grad_W2 != 0)*100:.1f}%")


## 10. 경사 하강법 구현

모든 구성 요소를 결합하여 전체 훈련 과정을 구현합니다.


In [ ]:
def gradient_descent(data, word2Ind, N, V, num_iters, alpha=0.03):
    """
    경사 하강법을 사용한 CBOW 모델 훈련
    
    Args:
        data: 전처리된 텍스트 데이터
        word2Ind: 단어→인덱스 딕셔너리
        N: 임베딩 차원
        V: 어휘집 크기
        num_iters: 훈련 반복 횟수
        alpha: 학습률
    
    Returns:
        W1, W2, b1, b2: 훈련된 모델 파라미터
    """
    # 모델 파라미터 초기화
    W1, W2, b1, b2 = initialize_model(N, V, random_seed=8855)
    
    # 훈련 설정
    batch_size = 128
    C = 2  # 컨텍스트 윈도우 크기
    iters = 0
    costs = []  # 비용 기록용
    
    print(f"훈련 시작:")
    print(f"  학습률: {alpha}")
    print(f"  배치 크기: {batch_size}")
    print(f"  컨텍스트 윈도우: {C}")
    print(f"  총 반복 횟수: {num_iters}")
    print("-" * 50)
    
    # 훈련 루프
    for x, y in get_batches(data, word2Ind, V, C, batch_size):
        # 순전파
        z, h = forward_prop(x, W1, W2, b1, b2)
        yhat = softmax(z)
        
        # 비용 계산
        cost = compute_cost(y, yhat, batch_size)
        costs.append(cost)
        
        # 주기적으로 비용 출력
        if (iters + 1) % 10 == 0:
            print(f"반복 {iters+1:3d}: 비용 = {cost:.6f}")
        
        # 역전파
        grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(
            x, yhat, y, h, W1, W2, b1, b2, batch_size
        )
        
        # 파라미터 업데이트
        W1 = W1 - alpha * grad_W1
        W2 = W2 - alpha * grad_W2
        b1 = b1 - alpha * grad_b1
        b2 = b2 - alpha * grad_b2
        
        iters += 1
        
        # 종료 조건
        if iters == num_iters:
            break
            
        # 학습률 스케줄링 (매 100회마다 감소)
        if iters % 100 == 0:
            alpha *= 0.66
            print(f"    → 학습률을 {alpha:.4f}로 조정")
    
    print("-" * 50)
    print(f"훈련 완료! 총 {iters}회 반복")
    print(f"최종 비용: {costs[-1]:.6f}")
    print(f"초기 비용: {costs[0]:.6f}")
    print(f"비용 개선: {costs[0] - costs[-1]:.6f}")
    
    return W1, W2, b1, b2, costs

print("경사 하강법 함수가 정의되었습니다.")


## 11. 모델 훈련 실행

실제로 CBOW 모델을 훈련시켜보겠습니다.


In [ ]:
# 훈련 파라미터 설정
C = 2  # 컨텍스트 윈도우 크기
N = 50  # 임베딩 차원
num_iters = 150  # 반복 횟수 (빠른 실행을 위해 원본보다 적게)

print("🚀 CBOW 모델 훈련을 시작합니다!")
print(f"데이터 크기: {len(data):,} 토큰")
print(f"어휘집 크기: {V:,} 단어")
print(f"임베딩 차원: {N}")
print(f"컨텍스트 윈도우: ±{C}")

# 단어-인덱스 딕셔너리 재생성 (정확성 확보)
word2Ind, Ind2word = get_dict(data)
V = len(word2Ind)

print(f"\\n훈련 시작...")
import time
start_time = time.time()

# 모델 훈련
W1_trained, W2_trained, b1_trained, b2_trained, training_costs = gradient_descent(
    data, word2Ind, N, V, num_iters
)

end_time = time.time()
training_time = end_time - start_time

print(f"\\n✅ 훈련 완료!")
print(f"훈련 시간: {training_time:.1f}초")
print(f"최종 파라미터 크기:")
print(f"  W1: {W1_trained.shape}")
print(f"  W2: {W2_trained.shape}")
print(f"  b1: {b1_trained.shape}")
print(f"  b2: {b2_trained.shape}")


## 12. 훈련 과정 시각화

훈련 중 비용의 변화를 시각화하여 학습 과정을 분석합니다.


In [ ]:
# 훈련 과정 시각화
plt.figure(figsize=(12, 5))

# 전체 비용 변화
plt.subplot(1, 2, 1)
plt.plot(training_costs, 'b-', linewidth=2, alpha=0.7)
plt.title('전체 훈련 과정의 비용 변화', fontsize=14, pad=20)
plt.xlabel('반복 횟수')
plt.ylabel('Cross-entropy 손실')
plt.grid(True, alpha=0.3)

# 최근 50회의 비용 변화 (세부 관찰)
plt.subplot(1, 2, 2)
if len(training_costs) > 50:
    plt.plot(training_costs[-50:], 'r-', linewidth=2, alpha=0.7)
    plt.title('최근 50회 반복의 비용 변화', fontsize=14, pad=20)
    plt.xlabel('반복 횟수 (최근 50회)')
else:
    plt.plot(training_costs, 'r-', linewidth=2, alpha=0.7)
    plt.title('전체 비용 변화', fontsize=14, pad=20)
    plt.xlabel('반복 횟수')
plt.ylabel('Cross-entropy 손실')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 훈련 통계 출력
print(f"📊 훈련 통계:")
print(f"  초기 비용: {training_costs[0]:.6f}")
print(f"  최종 비용: {training_costs[-1]:.6f}")
print(f"  비용 감소: {training_costs[0] - training_costs[-1]:.6f}")
print(f"  감소율: {((training_costs[0] - training_costs[-1]) / training_costs[0] * 100):.2f}%")

# 비용 변화 추세 분석
if len(training_costs) > 10:
    recent_trend = np.mean(training_costs[-10:]) - np.mean(training_costs[-20:-10])
    if recent_trend < 0:
        print(f"  최근 추세: 👍 계속 감소 중 ({recent_trend:.6f})")
    else:
        print(f"  최근 추세: 📈 증가 또는 안정화 ({recent_trend:.6f})")

print(f"\\n학습이 성공적으로 완료되었습니다! 이제 임베딩을 추출할 준비가 되었습니다.")


## 13. 단어 임베딩 추출

훈련된 모델에서 단어 임베딩을 추출합니다.


In [ ]:
# 단어 임베딩 추출
# W1과 W2의 평균을 사용 (일반적인 Word2Vec 접근법)
embeddings = (W1_trained.T + W2_trained) / 2.0

print(f"추출된 임베딩:")
print(f"  크기: {embeddings.shape}")
print(f"  각 단어는 {embeddings.shape[1]}차원 벡터로 표현됩니다")

# 특정 단어들의 임베딩 확인
test_words = ["king", "queen", "lord", "man", "woman", "prince", "ophelia", "rich", "happy"]
print(f"\\n분석할 단어들: {test_words}")

# 단어들이 어휘집에 있는지 확인
available_words = []
word_indices = []

for word in test_words:
    if word in word2Ind:
        available_words.append(word)
        word_indices.append(word2Ind[word])
        print(f"  ✓ '{word}' → 인덱스 {word2Ind[word]}")
    else:
        print(f"  ✗ '{word}' → 어휘집에 없음")

print(f"\\n분석 가능한 단어 수: {len(available_words)}/{len(test_words)}")

if len(available_words) > 0:
    # 선택된 단어들의 임베딩 추출
    X = embeddings[word_indices, :]
    print(f"\\n선택된 단어들의 임베딩 행렬 크기: {X.shape}")
    
    # 임베딩 벡터 통계
    print(f"\\n임베딩 통계:")
    print(f"  평균: {X.mean():.4f}")
    print(f"  표준편차: {X.std():.4f}")
    print(f"  최솟값: {X.min():.4f}")
    print(f"  최댓값: {X.max():.4f}")
    
    # 각 단어의 임베딩 벡터 일부 출력
    print(f"\\n각 단어의 임베딩 벡터 (처음 5차원):")
    for i, word in enumerate(available_words):
        embedding_sample = X[i, :5]
        print(f"  '{word}': [{', '.join([f'{x:.3f}' for x in embedding_sample])}, ...]")
else:
    print("⚠️  분석할 수 있는 단어가 없습니다. 다른 단어들을 시도해보세요.")
    
    # 어휘집에서 임의의 단어들 선택
    available_words = list(word2Ind.keys())[:9]  # 처음 9개 단어
    word_indices = [word2Ind[word] for word in available_words]
    X = embeddings[word_indices, :]
    print(f"\\n대신 다음 단어들로 분석합니다: {available_words}")
    print(f"임베딩 행렬 크기: {X.shape}")


## 14. PCA를 통한 임베딩 시각화

고차원 임베딩을 2차원으로 축소하여 시각화합니다.


In [ ]:
# PCA를 사용하여 2차원으로 축소
print("PCA를 사용하여 임베딩을 2차원으로 축소합니다...")

try:
    # PCA 적용
    result = compute_pca(X, 2)
    print(f"PCA 결과 크기: {result.shape}")
    print(f"축소된 좌표 범위:")
    print(f"  X축: {result[:, 0].min():.3f} ~ {result[:, 0].max():.3f}")
    print(f"  Y축: {result[:, 1].min():.3f} ~ {result[:, 1].max():.3f}")
    
    # 시각화
    plt.figure(figsize=(12, 8))
    plt.scatter(result[:, 0], result[:, 1], c='red', s=100, alpha=0.7, edgecolors='black')
    
    # 각 점에 단어 레이블 추가
    for i, word in enumerate(available_words):
        plt.annotate(word, 
                    xy=(result[i, 0], result[i, 1]), 
                    xytext=(5, 5), 
                    textcoords='offset points',
                    fontsize=12, 
                    fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
    
    plt.title('CBOW 모델로 학습한 단어 임베딩 (PCA 시각화)', fontsize=16, pad=20)
    plt.xlabel('주성분 1', fontsize=14)
    plt.ylabel('주성분 2', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # 축 범위를 조정하여 레이블이 잘리지 않도록
    x_margin = (result[:, 0].max() - result[:, 0].min()) * 0.1
    y_margin = (result[:, 1].max() - result[:, 1].min()) * 0.1
    plt.xlim(result[:, 0].min() - x_margin, result[:, 0].max() + x_margin)
    plt.ylim(result[:, 1].min() - y_margin, result[:, 1].max() + y_margin)
    
    plt.tight_layout()
    plt.show()
    
    # 단어 간 거리 분석
    print("\\n📏 단어 간 유클리드 거리 (원본 임베딩 공간):")
    from scipy.spatial.distance import pdist, squareform
    
    # 원본 고차원 임베딩에서의 거리
    distances = pdist(X, metric='euclidean')
    distance_matrix = squareform(distances)
    
    # 가장 유사한 단어 쌍들 (거리가 가까운)
    upper_triangle = np.triu(distance_matrix, k=1)
    min_dist_idx = np.unravel_index(np.argmax(upper_triangle == 0), upper_triangle.shape)
    
    print("가장 가까운 단어 쌍들:")
    sorted_indices = np.argsort(distances)
    for i in range(min(3, len(sorted_indices))):
        idx = sorted_indices[i]
        # 거리 행렬에서 인덱스 계산
        n = len(available_words)
        row = int((-1 + np.sqrt(1 + 8*idx)) / 2)
        col = int(idx - row * (row + 1) / 2 + row + 1)
        if row < len(available_words) and col < len(available_words):
            distance = distances[idx]
            print(f"  {available_words[row]} ↔ {available_words[col]}: {distance:.3f}")

except Exception as e:
    print(f"❌ PCA 시각화 중 오류 발생: {e}")
    print("대신 단순한 통계 분석을 제공합니다.")
    
    # 단어별 임베딩 크기(norm) 비교
    norms = np.linalg.norm(X, axis=1)
    print("\\n단어별 임베딩 벡터 크기(L2 norm):")
    for i, word in enumerate(available_words):
        print(f"  '{word}': {norms[i]:.3f}")


## 15. 임베딩 품질 분석

학습된 임베딩의 품질을 다양한 방법으로 분석해봅시다.


In [ ]:
# 임베딩 품질 분석

print("🔍 임베딩 품질 분석:")
print("=" * 50)

# 1. 코사인 유사도 분석
def cosine_similarity(a, b):
    """두 벡터 간의 코사인 유사도 계산"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

if len(available_words) >= 2:
    print("\\n1. 단어 간 코사인 유사도:")
    for i in range(min(3, len(available_words))):
        for j in range(i+1, min(3, len(available_words))):
            similarity = cosine_similarity(X[i], X[j])
            print(f"   '{available_words[i]}' ↔ '{available_words[j]}': {similarity:.4f}")

# 2. 가장 유사한/다른 단어 찾기
def find_most_similar_words(target_word, embeddings, word_list, word2ind, top_k=5):
    """특정 단어와 가장 유사한 단어들 찾기"""
    if target_word not in word2ind:
        return []
    
    target_idx = word2ind[target_word]
    target_embedding = embeddings[target_idx]
    
    similarities = []
    for word in word_list:
        if word != target_word:
            word_idx = word2ind[word]
            word_embedding = embeddings[word_idx]
            similarity = cosine_similarity(target_embedding, word_embedding)
            similarities.append((word, similarity))
    
    # 유사도 순으로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

print("\\n2. 단어 유사도 순위:")
for target_word in available_words[:3]:  # 처음 3개 단어에 대해
    similar_words = find_most_similar_words(target_word, embeddings, available_words, word2Ind)
    print(f"   '{target_word}'와 유사한 단어들:")
    for word, sim in similar_words:
        print(f"     {word}: {sim:.4f}")

# 3. 임베딩 공간의 분산 분석
print("\\n3. 임베딩 공간 분석:")
print(f"   전체 어휘집 크기: {embeddings.shape[0]:,} 단어")
print(f"   임베딩 차원: {embeddings.shape[1]}")

# 각 차원의 분산
dimension_variances = np.var(embeddings, axis=0)
print(f"   차원별 분산 - 평균: {dimension_variances.mean():.4f}, 표준편차: {dimension_variances.std():.4f}")
print(f"   가장 분산이 큰 차원: {np.argmax(dimension_variances)} (분산: {dimension_variances.max():.4f})")
print(f"   가장 분산이 작은 차원: {np.argmin(dimension_variances)} (분산: {dimension_variances.min():.4f})")

# 4. 임베딩 벡터들의 길이 분포
embedding_norms = np.linalg.norm(embeddings, axis=1)
print(f"\\n4. 임베딩 벡터 크기 분포:")
print(f"   평균 크기: {embedding_norms.mean():.4f}")
print(f"   표준편차: {embedding_norms.std():.4f}")
print(f"   최솟값: {embedding_norms.min():.4f}")
print(f"   최댓값: {embedding_norms.max():.4f}")

# 5. 학습의 효과 확인
print("\\n5. 학습 효과 검증:")
# 초기 랜덤 임베딩과 비교
W1_random, W2_random, _, _ = initialize_model(N, V, random_seed=42)
random_embeddings = (W1_random.T + W2_random) / 2.0
random_norms = np.linalg.norm(random_embeddings, axis=1)

print(f"   학습된 임베딩 평균 크기: {embedding_norms.mean():.4f}")
print(f"   랜덤 임베딩 평균 크기: {random_norms.mean():.4f}")
print(f"   차이: {abs(embedding_norms.mean() - random_norms.mean()):.4f}")

# 임베딩이 의미적으로 구조화되었는지 확인
print("\\n💡 임베딩 학습이 완료되었습니다!")
print("   - 유사한 단어들이 가까운 위치에 배치되었는지 확인해보세요")
print("   - 단어 간 코사인 유사도를 통해 의미적 관계를 파악할 수 있습니다")
print("   - PCA 시각화를 통해 고차원 임베딩의 2차원 투영을 관찰할 수 있습니다")


## 결론

이 노트북에서 우리는:

1. **CBOW 모델의 전체 구조**를 이해하고 구현했습니다
2. **신경망의 핵심 구성 요소들**을 단계별로 구현했습니다:
   - 가중치 초기화
   - 순전파 (Forward Propagation)
   - Softmax 활성화 함수
   - Cross-entropy 비용 함수
   - 역전파 (Backpropagation)
   - 경사 하강법 최적화

3. **실제 텍스트 데이터로 모델을 훈련**하여 의미있는 단어 임베딩을 학습했습니다

4. **학습된 임베딩을 분석하고 시각화**하여 품질을 평가했습니다

### 주요 학습 포인트:

- **CBOW의 핵심 아이디어**: 주변 단어들로 중심 단어를 예측
- **신경망 기반 언어 모델**: 확률적 언어 모델을 신경망으로 구현
- **임베딩 학습**: 희소한 원핫 벡터를 밀집한 의미 벡터로 변환  
- **최적화 과정**: 그래디언트 기반 학습의 실제 적용
- **고차원 데이터 시각화**: PCA를 통한 차원 축소와 해석

### CBOW vs 다른 방법들:

- **N-gram 모델**: 이산적 확률 vs 연속적 벡터 표현
- **Naive Bayes**: 단순 분류 vs 복합적 표현 학습
- **마르코프 체인**: 순차적 생성 vs 컨텍스트 기반 예측

CBOW 모델은 현대 NLP의 기초가 되는 **Word2Vec**의 핵심 아이디어를 보여주며, 트랜스포머와 같은 더 복잡한 모델들의 이해를 위한 중요한 디딤돌 역할을 합니다!

### 다음 단계:
- Skip-gram 모델과의 비교
- 더 큰 데이터셋으로의 확장
- 사전 훈련된 임베딩 (GloVe, FastText) 활용
- 트랜스포머 기반 모델로의 발전
